In [3]:
import sys, os
sys.path.append('/cr/users/filip/bin/')

from utils.Auger.SD import Monitoring

from utils.binaries import *
from utils.plotting import *

In [ ]:
thresholds = np.round(np.arange(1.0, 5.01, 0.05), 2)
data = pd.read_csv('/cr/tempdata01/filip/SSDCalib/BootstrapHistos/rates.txt', sep=' ', index_col=False)

data = data.dropna()
# data = data.drop(data[data['#id'] == 734].index)
# data = data.drop(data[data['#id'] == 1732].index)

# in UUB RANDOMS STATIONS FROM {Dec, Mar, Jun, Sep} 2021-2023
# 29681 events in dataset before fitting algorithm rework
# 44958 events in dataset AFTER fitting algoritm reworked

# in ALL STATIONS in {Jun, Dec} 2023
# 869_609 showers

In [ ]:
data

In [ ]:
from scipy.stats import zscore

# remove station specific outliers (3 sigma)
data_wo_outliers = pd.DataFrame([], columns=data.columns)
for station in np.unique(data['#id']):
    mask = data['#id'] == station
    df = data[mask]
    n_orig = len(df)

    for col in df.columns:
        if not col.endswith('xmip'): continue
        df = df.drop(df[np.abs(zscore(df[col])) > 3].index)
    
    n_wo_outlier = len(df)
    data_wo_outliers = pd.concat([data_wo_outliers, df])

    print(f"{n_orig - n_wo_outlier} outliers removed for station {station}...")

In [ ]:
so.jointplot(data_wo_outliers, x='mip', y='5.0xmip', palette='tab10')
plt.xlim(10, 60)
plt.ylim(0, 7)
plt.legend(title='Station ID')
plt.xlabel('$\mathrm{MIP}_\mathrm{Peak}$ / ADC')
plt.ylabel('Rate above $1.75\,I^\mathrm{SSD}_\mathrm{histo}$ / Hz')

In [ ]:
thresholds = np.round(np.arange(1.0, 5.01, 0.05), 2)
ax = plt.gca()

_ = plt.violinplot([data_wo_outliers[f"{t}xmip"].values for t in thresholds], 
                   positions=thresholds, 
                   widths=0.05,
                   showmeans=True)

rates = np.array([data_wo_outliers[f"{t}xmip"].mean() for t in thresholds])
sigma_sys = np.array([data_wo_outliers[f"{t}xmip"].std() for t in thresholds]) / rates
sigma_stat = 1/np.sqrt(rates * 61)
sigma = np.sqrt(sigma_sys ** 2 + sigma_stat ** 2)

error = ax.twinx()
error.plot(thresholds, sigma_stat, label=r"$\sigma_\mathrm{stat.}$", c='r', ls='--')
error.plot(thresholds, sigma_sys, label=r"$\sigma_{\mathrm{sys.}}$", c='b', ls=':')
error.plot(thresholds, sigma, label=r"$\sqrt{\Sigma_i\,\sigma_i^2}$", c='g', ls='-.')

ax.plot([], [], label=r"$\sigma_\mathrm{stat.}$")
ax.plot([], [],label=r"$\sigma_{\mathrm{sys.}}$")
ax.plot([], [], label=r"$\sqrt{\Sigma_i\,\sigma_i^2}$")
ax.errorbar([],[], [], label='Events', fmt='ko', ls=None, capsize=2)
ax.set_xlabel("Threshold / $I^\mathrm{SSD}_\mathrm{histo}$")
ax.set_ylabel("Rate above threshold $r^+$ / Hz")
ax.legend(loc='upper right', ncol=2)
error.set_ylabel(r'$\sigma_{r^+}\,/\,r^+$')
error.set_ylim(0, 0.5)

In [ ]:
# calc mean rate
mean_rates = np.zeros_like(thresholds)
stations = np.unique(data['#id'])
for i,t in enumerate(thresholds):
    for station in stations:
        df = data[data['#id'] == station]
        mean_rates[i] += df[f"{t}xmip"].mean() / len(stations)

with open('/cr/users/filip/Data/SDMonitHistos/mean_rates.txt', 'w') as f:
    for t, r in zip(thresholds, mean_rates):
        # print(t, np.round(r, 3))
        f.write(f"{t} {r:.3f}\n")